# Introduction 

This note evaluate ft-smollm 135m model with llama.cpp

# Env and tools prepare

In [1]:
!cp -r /kaggle/input/llama-cpp-binary-compiled-with-gpu/llama.cpp /kaggle/working/

In [2]:
!chmod +x -R llama.cpp
%cd llama.cpp
!cp -r /usr/local/cuda-12.3/targets /usr/local/nvidia/ 

/kaggle/working/llama.cpp


# Run perplexity

Perplexity is the main evaluation metric for large language models (LLMs). It measures how well the model predicts a given sequence of tokens.
Formally, the perplexity of LLMs is the exponentiated average negative log-likelihood,  which means **perplexity smaller is better** 

Perplexity benchmark shows how accurate the model is 

In [3]:

!/kaggle/working/llama.cpp/llama-perplexity -m  /kaggle/input/ft-smollm-135m-instruct-on-hf-ultrafeedback-gguf/gguf/fp16/1/ft-smollm-135M-instruct-on-hf-ultrafeedback-f16.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > ppl_fp16.log 2>&1
!/kaggle/working/llama.cpp/llama-perplexity -m  /kaggle/input/ft-smollm-135m-instruct-on-hf-ultrafeedback-gguf/gguf/q4/1/ft-smollm-135M-instruct-on-hf-ultrafeedback-Q4_K_M-v2.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > ppl_Q4_v2.log 2>&1
!/kaggle/working/llama.cpp/llama-perplexity -m  /kaggle/input/ft-smollm-135m-instruct-on-hf-ultrafeedback-gguf/gguf/q4/1/ft-smollm-135M-instruct-on-hf-ultrafeedback-Q4_K_M.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > ppl_Q4.log 2>&1


In [4]:
!tail -6 ppl_fp16.log
!tail -6 ppl_Q4_v2.log
!tail -6 ppl_Q4.log

Final estimate: PPL = 42.4663 +/- 0.67752

llama_perf_context_print:        load time =    2574.87 ms
llama_perf_context_print: prompt eval time =   37989.66 ms / 131072 tokens (    0.29 ms per token,  3450.20 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   57647.41 ms / 131073 tokens
Final estimate: PPL = 50.7943 +/- 0.82164

llama_perf_context_print:        load time =    1200.24 ms
llama_perf_context_print: prompt eval time =   28016.71 ms / 131072 tokens (    0.21 ms per token,  4678.35 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   47563.67 ms / 131073 tokens
Final estimate: PPL = 50.7943 +/- 0.82164

llama_perf_context_print:        load time =    1246.01 ms
llama_perf_context_print: prompt eval 

# Benchmarking the Inference Throughput and Memory Consumption 

Llama-bench shows performance and resource consumption. 
There are three types of test
- Prompt processing (pp): processing a prompt in batches (-p)
- Text generation (tg): generating a sequence of tokens (-n)
- Prompt processing + text generation (pg): processing a prompt followed by generating a sequence of tokens (-pg)

In the following test, we run text generation and prompt processing


In [5]:
!/kaggle/working/llama.cpp/llama-bench -m /kaggle/input/ft-smollm-135m-instruct-on-hf-ultrafeedback-gguf/gguf/fp16/1/ft-smollm-135M-instruct-on-hf-ultrafeedback-f16.gguf -m /kaggle/input/ft-smollm-135m-instruct-on-hf-ultrafeedback-gguf/gguf/q4/1/ft-smollm-135M-instruct-on-hf-ultrafeedback-Q4_K_M.gguf -m /kaggle/input/ft-smollm-135m-instruct-on-hf-ultrafeedback-gguf/gguf/q4/1/ft-smollm-135M-instruct-on-hf-ultrafeedback-Q4_K_M-v2.gguf -mg 1 -ngl 10,20,30,35 -n 128,256,512 

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: Tesla P100-PCIE-16GB, compute capability 6.0, VMM: yes
| model                          |       size |     params | backend    | ngl |   main_gpu |          test |                  t/s |
| ------------------------------ | ---------: | ---------: | ---------- | --: | ---------: | ------------: | -------------------: |
| llama ?B F16                   | 310.63 MiB |   162.83 M | CUDA       |  10 |          1 |         pp512 |      6049.36 ± 80.80 |
| llama ?B F16                   | 310.63 MiB |   162.83 M | CUDA       |  10 |          1 |         tg128 |         79.87 ± 2.05 |
| llama ?B F16                   | 310.63 MiB |   162.83 M | CUDA       |  10 |          1 |         tg256 |         79.42 ± 0.40 |
| llama ?B F16                   | 310.63 MiB |   162.83 M | CUDA       |  10 |          1 |         tg512 |         75.73 ± 0.93 |
| llama ?B F16   